# Produce False Colour Geotiffs Iteratively

**What does this notebook do?** 


**Date** March 2018

**Author** Claire Krause

**Tags** 

## This is the bit that allows you to call in a stand alone python file. 

The two python files `DEAPlotting.py` and `DEADataHandling.py` are both sitting in the `algorithms` folder.

In [ ]:
% pylab notebook
% load_ext autoreload
% autoreload

from datacube import Datacube
from datetime import datetime
from IPython.display import clear_output

import sys
import os
sys.path.append(os.path.abspath('/g/data/r78/cek156/dea-notebooks/algorithms'))
import DEAPlotting
import DEADataHandling

dc = Datacube(app = 'test')
s2dc = Datacube(config='/home/156/cek156/Sentinel2_datacube.conf')

## User inputs - set up plot parameters

In [ ]:
AllSensors = ['ls8', 's2a', 's2b']
output_files = '/g/data/r78/Pretty_pictures/Roebuck/'

# Extents
latmin = -17.62047
latmax = -18.30511
lonmin = 122.27736
lonmax = 123.12217

# Years
start_year = 2016
end_year = 2019

# A percentage of pixels that can be missing
acceptable_missing_data = 0.1

# How would you like to process the data? By year (smaller areas), or by month (larger areas)?
#Timestep = 'year'
Timestep = 'month'

## Year at a time

In [ ]:
if Timestep == 'year':
    for Sensor in AllSensors:
        print('Working on {0}'.format(Sensor))
        for year in range(start_year, end_year):

        ## Landsat workflow
            if Sensor[0] == 'l':
                print('Working on {0} to {1}'.format(year, year))
                queryls = {'lat': (latmin, latmax),
                           'lon': (lonmin, lonmax),
                           'time': ('{0}-01-01'.format(year), '{0}-12-31'.format(year))
                           }
                try:
                    Alldata, LScrs, LSaffine = DEADataHandling.load_nbarx(dc, Sensor, queryls, 
                                                                          bands_of_interest = ['swir1','nir','green'], 
                                                                          product = 'nbar')

                        print('{0} timesteps loaded'.format(Alldata.time.count().values))
                        threshold = Alldata.x.count() * Alldata.y.count() * acceptable_missing_data
                        AlldataFiltered = Alldata.where(Alldata.green.isnull().sum(axis = (1, 2)) 
                                                        <= int(threshold)).dropna(dim = 'time', how = 'all')

                        print('{0} timesteps with less than {1}% missing pixels'.format(AlldataFiltered.time.count().values,
                                                                                      acceptable_missing_data * 100))

                        ## Draw a false colour image using our imported function and save
                        for ix, times in enumerate(AlldataFiltered.time):
                            outnameGTiff = '{0}{1}_{2}.tif'.format(output_files, str(times.time.values)[:10], Sensor)
                            outnamejpg = '{0}{1}_{2}.jpg'.format(output_files, str(times.time.values)[:10], Sensor)
                            plt, fig = DEAPlotting.three_band_image(AlldataFiltered.isel(time = ix), 
                                                                    ['swir1','nir','green'], 
                                                                    title = '')   
                            #DEADataHandling.dataset_to_geotiff(outnameGTiff, Alldata.isel(time = ix))
                            savefig(outnamejpg, dpi = 300)
                            plt.close("all")
                            close(fig)
                    except (TypeError, AttributeError):
                        print('No data available for this time step')
                        pass
                    clear_output()   

            ## Sentinel workflow
            if Sensor[0] == 's':
                print('Working on {0} to {1}'.format(year, year))
                try:
                    querys2 = {'output_crs': 'EPSG:3577',
                               'resolution': (-10, 10),
                               'lat': (latmin, latmax),
                                   'lon': (lonmin, lonmax),
                               'time': ('{0}-01-01'.format(year), '{0}-12-31'.format(year))
                               }
                    prodname = '{0}_ard_granule'.format(Sensor)
                    Alldata, LScrs, LSaffine = DEADataHandling.load_sentinel(s2dc, prodname, querys2, 
                                                                             filter_cloud = False,
                                                                             bands_of_interest = ['swir1','nir1', 
                                                                                                  'green'])

                    print('{0} timesteps loaded'.format(Alldata.time.count().values))
                    threshold = Alldata.x.count() * Alldata.y.count() * acceptable_missing_data
                    AlldataFiltered = Alldata.where(Alldata.green.isnull().sum(axis = (1, 2)) 
                                                    <= int(threshold)).dropna(dim = 'time', how = 'all')

                    print('{0} timesteps with less than {1}% missing pixels'.format(AlldataFiltered.time.count().values,
                                                                                  acceptable_missing_data * 100))

                    ## Draw a false colour image using our imported function and save
                    for ix, times in enumerate(AlldataFiltered.time):
                        outnameGTiff = '{0}{1}_{2}.tif'.format(output_files, str(times.time.values)[:10], Sensor)
                        outnamejpg = '{0}{1}_{2}.jpg'.format(output_files, str(times.time.values)[:10], Sensor)
                        plt, fig = DEAPlotting.three_band_image(AlldataFiltered.isel(time = ix), 
                                                                ['swir1','nir1','green'], 
                                                                title = '')   
                        #DEADataHandling.dataset_to_geotiff(outnameGTiff, Alldata.isel(time = ix))
                        savefig(outnamejpg, dpi = 300)
                        plt.close("all")
                        close(fig)
                except (TypeError, AttributeError):
                    print('No data available for this time step')
                    pass
                clear_output()   

## Month at a time
For larger areas.

In [ ]:
if Timestep == 'month':
    for Sensor in AllSensors:
        print('Working on {0}'.format(Sensor))
        for year in range(start_year, end_year):
            for month in range (1, 12):

            ## Landsat workflow
                if Sensor[0] == 'l':
                    print('Working on {0}/{1}'.format(month, year))
                    try:
                        queryls = {'lat': (latmin, latmax),
                                   'lon': (lonmin, lonmax),
                                   'time': ('{0}-{1}-01'.format(year, month), '{0}-{1}-01'.format(year, month + 1))
                                   }
                        prodname = '{0}_ard_granule'.format(Sensor)
                        Alldata, LScrs, LSaffine = DEADataHandling.load_nbarx(dc, Sensor, queryls, 
                                                                              bands_of_interest = ['swir1','nir','green'], 
                                                                              product = 'nbar',
                                                                              filter_pq = False)

                        print('{0} timesteps loaded'.format(Alldata.time.count().values))
                        threshold = Alldata.x.count() * Alldata.y.count() * acceptable_missing_data
                        AlldataFiltered = Alldata.where(Alldata.green.isnull().sum(axis = (1, 2)) 
                                                        <= int(threshold)).dropna(dim = 'time', how = 'all')

                        print('{0} timesteps with less than {1}% missing pixels'.format(AlldataFiltered.time.count().values,
                                                                                      acceptable_missing_data * 100))

                        ## Draw a false colour image using our imported function and save
                        for ix, times in enumerate(AlldataFiltered.time):
                            outnameGTiff = '{0}{1}_{2}.tif'.format(output_files, str(times.time.values)[:10], Sensor)
                            outnamejpg = '{0}{1}_{2}.jpg'.format(output_files, str(times.time.values)[:10], Sensor)
                            plt, fig = DEAPlotting.three_band_image(AlldataFiltered.isel(time = ix), 
                                                                    ['swir1','nir','green'], 
                                                                    title = '')   
                            #DEADataHandling.dataset_to_geotiff(outnameGTiff, Alldata.isel(time = ix))
                            savefig(outnamejpg, dpi = 300)
                            plt.close("all")
                            close(fig)
                    except (TypeError, AttributeError):
                        print('No data available for this time step')
                        pass
                    clear_output()   

                ## Sentinel workflow
                if Sensor[0] == 's':
                    print('Working on {0}/{1}'.format(month, year))
                    try:
                        querys2 = {'output_crs': 'EPSG:3577',
                                   'resolution': (-10, 10),
                                   'lat': (latmin, latmax),
                                   'lon': (lonmin, lonmax),
                                   'time': ('{0}-{1}-01'.format(year, month), '{0}-{1}-01'.format(year, month + 1))
                                   }
                        prodname = '{0}_ard_granule'.format(Sensor)
                        Alldata, LScrs, LSaffine = DEADataHandling.load_sentinel(s2dc, prodname, querys2, 
                                                                                 filter_cloud = False,
                                                                                 bands_of_interest = ['swir1','nir1',
                                                                                                      'green'])

                        print('{0} timesteps loaded'.format(Alldata.time.count().values))
                        threshold = Alldata.x.count() * Alldata.y.count() * acceptable_missing_data
                        AlldataFiltered = Alldata.where(Alldata.green.isnull().sum(axis = (1, 2)) 
                                                        <= int(threshold)).dropna(dim = 'time', how = 'all')

                        print('{0} timesteps with less than {1}% missing pixels'.format(AlldataFiltered.time.count().values,
                                                                                      acceptable_missing_data * 100))

                        ## Draw a false colour image using our imported function and save
                        for ix, times in enumerate(AlldataFiltered.time):
                            outnameGTiff = '{0}{1}_{2}.tif'.format(output_files, str(times.time.values)[:10], Sensor)
                            outnamejpg = '{0}{1}_{2}.jpg'.format(output_files, str(times.time.values)[:10], Sensor)
                            plt, fig = DEAPlotting.three_band_image(AlldataFiltered.isel(time = ix), ['swir1','nir1','green'], 
                                                                    title = '')   
                            #DEADataHandling.dataset_to_geotiff(outnameGTiff, Alldata.isel(time = ix))
                            savefig(outnamejpg, dpi = 300)
                            plt.close("all")
                            close(fig)
                    except (TypeError, AttributeError):
                        print('No data available for this time step')
                        pass
                    clear_output()   